## /* PART 2: SQLite

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
for the following questions.  



## My answer

Unfortunately, I couldn’t find a direct way to export the country club data from PHPMyAdmin into SQLite. 
As a result, I followed these steps:
1.	Downloaded each table from PHPMyAdmin in .json format (3 files, named after each table).
2.	Removed all comments from the downloaded files.
3.	Developed a Python script to create a SQLite database with 3 tables (Bookings, Facilities, and Members) 
	and to insert data from the downloaded .json files, using the Pandas DataFrame method to_sql.

Here is my Python script:

In [42]:
import sqlite3
import pandas as pd
import json

connection = sqlite3.connect("country_club_SQLite.db")
cursor = connection.cursor()
cursor.execute("CREATE TABLE IF NOT EXISTS 'Bookings' \
    ('bookid'   INTEGER, \
    'facid'     INTEGER, \
    'memid'     INTEGER, \
    'starttime' TEXT, \
    'slots'     INTEGER);")

cursor.execute("CREATE TABLE IF NOT EXISTS 'Members' \
    ('memid'    INTEGER, \
    'surname'   TEXT, \
    'firstname' TEXT, \
    'address'   TEXT, \
    'zipcode'   INTEGER, \
    'telephone' TEXT, \
    'recommendedby' TEXT, \
    'joindate'  TEXT);")

cursor.execute("CREATE TABLE IF NOT EXISTS 'Facilities' \
  ('facid' INTEGER, \
  'name' TEXT, \
  'membercost' NUMERIC, \
  'guestcost' NUMERIC, \
  'initialoutlay' INTEGER, \
  'monthlymaintenance' INTEGER);")

connection.commit()

tables = ['Bookings', 'Facilities', 'Members']

for table in tables:
    with open(table + ".json") as file:
        data = json.load(file)
    df = pd.DataFrame(data)
    df.to_sql(table, connection, if_exists='append', index=False)
    connection.commit()

### However, after that I connected to the database which was provided with Tier Two package.
...but both databases give the same result

In [48]:
database_file = "sqlite_db_pythonsqlite.db"         # database provided with the task
# database_file = "country_club_SQLite.db"          # my SQLite database

## QUESTIONS:
/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

In [49]:
import sqlite3
import pandas as pd

connection = sqlite3.connect(database_file)

my_query = "\
    SELECT \
        f.name,  \
        SUM(CASE WHEN b.memid = 0 THEN f.guestcost * b.slots \
                ELSE f.membercost * b.slots END) AS revenue\
    FROM Bookings AS b\
    LEFT JOIN Facilities AS f\
    ON b.facid = f.facid \
    GROUP BY f.name \
    ORDER BY revenue"

df = pd.read_sql_query(my_query, connection)
print(df)

              name  revenue
0     Table Tennis    180.0
1    Snooker Table    240.0
2       Pool Table    270.0
3  Badminton Court   1906.5
4     Squash Court  13468.0
5   Tennis Court 1  13860.0
6   Tennis Court 2  14310.0
7   Massage Room 2  14454.6
8   Massage Room 1  50351.6



/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */

In [50]:
import sqlite3
import pandas as pd

connection = sqlite3.connect(database_file)

my_query = "\
    SELECT \
        m.surname, \
        m.firstname, \
        r.surname AS rec_by_surname, \
        r.firstname AS rec_by_firstname\
    FROM Members AS m \
    LEFT JOIN Members AS r\
    ON r.memid = m.recommendedby \
    ORDER BY m.surname, m.firstname"

df = pd.read_sql_query(my_query, connection)
print(df)

              surname  firstname rec_by_surname rec_by_firstname
0               Bader   Florence       Stibbons           Ponder
1               Baker       Anne       Stibbons           Ponder
2               Baker    Timothy        Farrell           Jemima
3              Boothe        Tim         Rownam              Tim
4             Butters     Gerald          Smith           Darren
5              Coplin       Joan          Baker          Timothy
6             Crumpet      Erica          Smith            Tracy
7                Dare      Nancy       Joplette           Janice
8             Farrell      David           None             None
9             Farrell     Jemima           None             None
10              GUEST      GUEST           None             None
11            Genting    Matthew        Butters           Gerald
12               Hunt       John        Purview        Millicent
13              Jones      David       Joplette           Janice
14              Jones    

/* Q12: Find the facilities with their usage by member, but not guests */

In [51]:
import sqlite3
import pandas as pd

connection = sqlite3.connect(database_file)

my_query = "\
    SELECT \
        f.name, \
        COUNT(b.memid) AS usage_by_members \
    FROM Bookings AS b \
    INNER JOIN Facilities AS f \
    ON b.facid = f.facid \
    WHERE b.memid != 0 \
    GROUP BY f.name;"

df = pd.read_sql_query(my_query, connection)
print(df)

              name  usage_by_members
0  Badminton Court               344
1   Massage Room 1               421
2   Massage Room 2                27
3       Pool Table               783
4    Snooker Table               421
5     Squash Court               195
6     Table Tennis               385
7   Tennis Court 1               308
8   Tennis Court 2               276




/* Q13: Find the facilities usage by month, but not guests */

In [52]:
import sqlite3
import pandas as pd

connection = sqlite3.connect(database_file)

my_query = "\
	SELECT \
		f.name, \
		strftime('%m', DATE(b.starttime)) AS month, \
		COUNT(b.memid) AS usage_by_members \
	FROM Bookings AS b \
	INNER JOIN Facilities AS f \
	ON b.facid = f.facid \
	WHERE b.memid != 0 \
    GROUP BY f.name, month \
	ORDER BY f.name, month;"

df = pd.read_sql_query(my_query, connection)
print(df) 

               name month  usage_by_members
0   Badminton Court    07                51
1   Badminton Court    08               132
2   Badminton Court    09               161
3    Massage Room 1    07                77
4    Massage Room 1    08               153
5    Massage Room 1    09               191
6    Massage Room 2    07                 4
7    Massage Room 2    08                 9
8    Massage Room 2    09                14
9        Pool Table    07               103
10       Pool Table    08               272
11       Pool Table    09               408
12    Snooker Table    07                68
13    Snooker Table    08               154
14    Snooker Table    09               199
15     Squash Court    07                23
16     Squash Court    08                85
17     Squash Court    09                87
18     Table Tennis    07                48
19     Table Tennis    08               143
20     Table Tennis    09               194
21   Tennis Court 1    07       